# Read all data
as peaks [for artifact removal and most HRV metrics] and as RRI [for nonlinear HRV metrics] to be computed in neurokit2

In [ ]:
# call conda datanalysis environment
!conda init bash

In [ ]:
!conda activate datanalysis #or use your own preferred venv

In [105]:
# Do all package imports

import neurokit2 as nk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
from pathlib import Path
from scipy.stats import variation
from hmmlearn import hmm

In [9]:
# load Matlab data files
from scipy.io import loadmat

In [ ]:
os.getcwd()

In [6]:
# Import raw peaks from the mat files; adjust to fit your input data format

f_filepath_peaks=Path.cwd()/"raw_peaks/f" #fetal raw peaks mat files;
m_filepath_peaks=Path.cwd()/"raw_peaks/m" #maternal raw peaks mat files;

In [7]:
f_peaks_files = [f for f in sorted(f_filepath_peaks.iterdir())  #create a list of relevant files in directory
            if f.suffix == '.mat']
m_peaks_files = [f for f in sorted(m_filepath_peaks.iterdir())  #create a list of relevant files in directory
            if f.suffix == '.mat']

In [48]:
# Read one file at a time using the above list, trim, clean, convert to RRI
# The present syntax is for a specific ECG format; adopt to your use case
# Iterate over i files in the f_ or m_ peaks_files lists and extract the correct peaks channel as numpy array
# Proceed with the steps below: HRV compute, save

def read_mat_file(f_peaks_file, m_peaks_file):
    # Import 5th row of the mat file's peak data which has 1000 Hz sampling rate
    f_file_PEAK_raw=loadmat(f_peaks_file)
    m_file_PEAK_raw=loadmat(m_peaks_file)

    f_peaks=f_file_PEAK_raw['fetal_Rpeaks'][4] #this is my 5th row ECG-SAVER-extracted peaks channel
    m_peaks=m_file_PEAK_raw['mother_Rpeaks'][4] #this is my 5th row
    
    # Trim trailing zeros
    f_peaks_trimmed=np.trim_zeros(f_peaks,trim='b')
    m_peaks_trimmed=np.trim_zeros(m_peaks,trim='b')
    
    # Artifact removal [see next section for details]
    f_clean_peaks=nk.signal_fixpeaks(f_peaks_trimmed, sampling_rate=1000, iterative=False, show=False,interval_min=0.33,interval_max=0.75, method="kubios") #allow 80-180 bpm
    m_clean_peaks=nk.signal_fixpeaks(m_peaks_trimmed, sampling_rate=1000, iterative=False, show=False,interval_min=0.4,interval_max=1.5, method="kubios") #allow 40-150 bpm
    
    # Document artifacts from each run as clean_peaks_rri[0]: build a dataframe for each file over all segments
    
    # Convert to RRI
    f_rri = peaks_to_rri(f_clean_peaks[1], sampling_rate=1000, interpolate=False)
    m_rri = peaks_to_rri(m_clean_peaks[1], sampling_rate=1000, interpolate=False)
        
    return f_clean_peaks[1], m_clean_peaks[1], f_rri, m_rri, f_clean_peaks[0], m_clean_peaks[0]

In [12]:
# Some NK functions [clean peaks function, complexity HRV metrics] take RRIs 
# So use these UDFs borrowed from the NK package: convert peaks to RRI on the cleaned peaks output

def peaks_to_rri(peaks=None, sampling_rate=1000, interpolate=False, **kwargs):

    rri = np.diff(peaks) / sampling_rate * 1000

    if interpolate is False:
        return rri

    else:

        # Minimum sampling rate for interpolation
        if sampling_rate < 10:
            sampling_rate = 10

        # Compute length of interpolated heart period signal at requested sampling rate.
        desired_length = int(np.rint(peaks[-1]))

        rri = signal_interpolate(
            peaks[1:],  # Skip first peak since it has no corresponding element in heart_period
            rri,
            x_new=np.arange(desired_length),
            **kwargs
        )
        return rri, sampling_rate

# Artifact correction
Integrated into the above UDF red_mat_file

In [ ]:
# https://neurokit2.readthedocs.io/en/latest/functions.html#neurokit2.signal.signal_fixpeaks
# Artifact removal on peaks using Kubios: write into UDF taking trimmed_peaks input
# caution: nk.signal_fixpeaks takes peaks, not RRI! 

# nk.signal_fixpeaks saves the corrected peak locations to the [1] index of the output data sturcture
# accessible like so: clean_peaks[1]

# Review the settings for fetal versus maternal RRI inputs! Adjust to match your RRI physiology
# interval_min – minimum interval btw peaks | interval_max – maximum interval btw peaks.

f_clean_peaks=nk.signal_fixpeaks(f_peaks_trimmed, sampling_rate=1000, iterative=False, show=False,interval_min=0.1,interval_max=0.25, method="kubios")
m_clean_peaks=nk.signal_fixpeaks(m_peaks_trimmed, sampling_rate=1000, iterative=False, show=False,interval_min=0.1,interval_max=0.25, method="kubios")

In [ ]:
# Convert trimmed and cleaned peaks to RRI (using _trimmmed_ raw peaks as input!)
rri_clean = peaks_to_rri(clean_peaks_peaks[1], sampling_rate=1000, interpolate=False)

# Compute all HRV metrics segment-wise

In [51]:
# UDF compute_HRV
# This UDF computes all [regular and extra non-linear] HRV metrics segment-wise for a file

def compute_HRV(peaks,rri,SubjectID):
    
    # Regular HRV matrix (from peaks)
    
    duration_peaks=peaks[len(peaks)-1] #gives me the duration in samples

    divider=duration_peaks/1000/60/5  #sampling_rate, 5 min window segments
    segment=np.array_split(peaks,divider) #divide in segments of 5 min; the last segment may be shorter; discard during statistical analysis on HRV metrics

    hrv_segment_df=pd.DataFrame()

    for i in range(len(segment)):
        hrv_segment=nk.hrv(segment[i],sampling_rate=1000, show=False)
        hrv_segment_df = pd.concat([hrv_segment_df,hrv_segment],ignore_index=True)

    # Additional nonlinear HRV metrics from RRIs
    
    segment=np.array_split(rri,divider) #divide _RRI_ in segments of 5 min; the last segment may be shorter; discard during statistical analysis on HRV metrics

    #create my dataframe structure to which to append the list as a row in the following
    hrv_extra_columns=['optimal time delay','FuzzEn','FuzzEnMSE','FuzzEnRCMSE','cApEn','segment duration, s','SubjectID']
    hrv_extra_complexity_df=pd.DataFrame(columns=hrv_extra_columns)
    df_length=len(hrv_extra_complexity_df)
    hrv_extra_complexity_df_total=pd.DataFrame(columns=hrv_extra_columns)

    for i in range(len(segment)):
        optimal_complexity_parameters = nk.complexity_delay(segment[i], delay_max=100, method='fraser1986', show=False)
        hrv_extra_complexity_segment_fuzen=nk.entropy_fuzzy(segment[i], delay=optimal_complexity_parameters)
        hrv_extra_complexity_segment_fuzen_mse=nk.complexity_fuzzymse(segment[i],fuzzy=True)
        hrv_extra_complexity_segment_fuzen_rcmse=nk.complexity_fuzzyrcmse(segment[i], fuzzy=True, composite=True, refined=True) 
        hrv_extra_complexity_segment_capen=nk.entropy_approximate(segment[i], delay=optimal_complexity_parameters, corrected=True)
        segment_duration=np.sum(segment[i])/1000 #segment duration in seconds
        #join all individual output floats including values of segment[i] - i.e., for each segment - and its duration in seconds as numpy.sum(segment[1])/1000 
        hrv_extra_complexity = [optimal_complexity_parameters, hrv_extra_complexity_segment_fuzen,hrv_extra_complexity_segment_fuzen_mse,hrv_extra_complexity_segment_fuzen_rcmse,hrv_extra_complexity_segment_capen,segment_duration,SubjectID]
        hrv_extra_complexity_df.loc[df_length]=hrv_extra_complexity
        hrv_extra_complexity_df_total = pd.concat([hrv_extra_complexity_df_total,hrv_extra_complexity_df],ignore_index=True)

    # simply concatenate both df's horizontally; this scales allowing addition of other df's from bivariate computations
    
    hrv_final_df=pd.concat([hrv_segment_df, hrv_extra_complexity_df_total],axis=1)

    return hrv_final_df #this is per subject with SubjectID output along on the right side

# Compute higher order HRV metrics
This includes statistical and HMM estimates

In [308]:
def compute_basic_stats(ts_data, SubjectID):

    # compute mean and variation
    # assuming "ts_data" is where my HRV metric values list is per subject
    
    HRV_mean=np.mean(ts_data.values.tolist())
    
    HRV_coeff_variation=variation(ts_data.values.tolist())
    # this function works similar to variation() but works purely with numpy
    # cv = lambda x: np.std(x) / np.mean(x)

    # First quartile (Q1) 
    Q1 = np.percentile(ts_data, 25, interpolation = 'midpoint') 
    # Third quartile (Q3) 
    Q3 = np.percentile(ts_data, 75, interpolation = 'midpoint') 
    # Interquaritle range (IQR) 
    IQR = Q3 - Q1 
    midhinge = (Q3 + Q1)/2
    quartile_coefficient_dispersion = (IQR/2)/midhinge
    
    # adding entropy estimate; this is experimental!
    # HRV_ts_entropy=nk.entropy_sample(ts_data) 
    # yielding error "could not broadcast input array from shape (7,1) into shape (7)" | the following syntax fixes that and is more elegant in that it estimates optimal delay
    # optimal_complexity_parameters = nk.complexity_delay(ts_data.to_numpy, delay_max=6, method='fraser1986', show=False)
    # HRV_ts_entropy=nk.entropy_fuzzy(ts_data.to_numpy, delay=optimal_complexity_parameters)
    # still yielding len error 
    HRV_ts_entropy=nk.entropy_shannon(ts_data)
        
    basic_stats=[SubjectID, HRV_mean, HRV_coeff_variation[0], quartile_coefficient_dispersion, HRV_ts_entropy]
    return basic_stats

In [277]:
#HMM Model

def do_hmm(ts_data):
    
    #ts_data=numpy.array(data)
    gm = hmm.GaussianHMM(n_components=2)
    gm.fit(ts_data.reshape(-1, 1))
    hmm_states = gm.predict(ts_data.reshape(-1, 1))
    #hmm_states=[states.tolist()]
    print(hmm_states)
    
    return hmm_states # next, add _states_ iteratively for all subjects to states_Uber list to spot patterns

In [185]:
# deal with last column which is string and needs to be skipped

def skip_last_column(lst):
    # unpack the list of lists
    def Extract(lst):
        return [item[0] for item in lst]
    # check for string in the first sublist (all I need to decide to skip it for numpy operations)
    element_to_check=Extract(lst)[0]
    return isinstance(element_to_check, str) #return Boolean for presence of string in the sublist

In [278]:
def compute_higher_HRV(hrv_final_df, SubjectID): 
    
    # assuming "hrv_final_df" is the dataframe where the HRV metric values are listed segment-wise per subject
    
    # compute basic stats
    
    higher_order_HRV_basic_stats=[]
    
    for i in range(hrv_final_df.shape[1]): #last column is the SubjectID string, so skipping it below
        
        hrv_metric=hrv_final_df.iloc[:,[i]].values
        
        #String skip logic to skip over SubjectID column
        if skip_last_column(hrv_final_df.iloc[:,[i]].values) == False:
            HRV_results_temp1=compute_basic_stats(hrv_final_df.iloc[:,[i]].astype(np.float64),SubjectID)
            higher_order_HRV_basic_stats.append(HRV_results_temp1)
    
        else:
            i+=1

    HRV_basic_stats=pd.DataFrame(higher_order_HRV_basic_stats, columns=['SubjectID','HRV_mean', 'HRV_coeff_variation', 'quartile_coefficient_dispersion','HRV metrics entropy'])
    hrv_columns=hrv_final_df.columns[0:63] #make sure I don't select the last column which has SubjectID
    HRV_basic_stats.index=[hrv_columns]
    HRV_basic_stats_final=HRV_basic_stats.T #transpose
    
    # compute HMM stats: computing on just 7 data points leads to errors in some instances, so omit for now and revisit later when used on longer HRV metrics time series, say, several hours 
    
    # Estimate HMM probabilities output for a given segmented HRV metric
    # Then compute basic_stats on this estimate;
    # Hypothesis: stable tracings will have tight distributions of HMM values and resemble entropy estimates;
    # This will apply statistically significantly for physiologically stressed (tighter distributions) versus control subjects

    #higher_order_HRV_basic_stats_on_HMM=[]
    
    
    #for i in range(hrv_final_df.shape[1]): #last column is the SubjectID string, so removing it
        
    #    hrv_metric=hrv_final_df.iloc[:,[i]].values
    #    print("HRV_metric has the type", type(hrv_metric))
        # some HRV metrics have NaNs and the "do_hmm" script crashes on those; 
        # Adding logic to skip if NaN is present
    #    a=any(pd.isna(hrv_metric)) #checking if _any_ values in HRV metrics list are NaN
    #    b=skip_last_column(hrv_metric)
    #    skip_reasons={a:'True', b:'True'}
        #NaN or string skip logic
    #    if any(skip_reasons):
    #        i+=1
    #    else:
    #        HRV_results_hmm_temp2=do_hmm(hrv_metric)
    #        print(HRV_results_hmm_temp2)
    #        print(type(HRV_results_hmm_temp2))
    #        HRV_results_stats_hmm_temp=compute_basic_stats(HRV_results_hmm_temp2.tolist(),SubjectID) #j being the file number; != SubjectID
    #        higher_order_HRV_basic_stats_on_HMM.append(HRV_results_stats_hmm_temp)
    
    #HRV_basic_stats_on_HMM=pd.DataFrame(higher_order_HRV_basic_stats_on_HMM, columns=['HRV_HMM_mean', 'HRV_HMM_coeff_variation', 'HMM_quartile_coefficient_dispersion','HMM_HRV metrics entropy'])
    #HRV_basic_stats_on_HMM.index=[hrv_columns]
    #HRV_basic_stats_on_HMM_final=HRV_basic_stats_on_HMM.T                  #transpose
        
    #higher_hrv_final_df=pd.concat([HRV_basic_stats_final, HRV_basic_stats_on_HMM_final],axis=1)    
    
    higher_hrv_final_df=HRV_basic_stats_final #leaving the syntax above for when the data allow HMM analysis    
        
    return higher_hrv_final_df #this includes SubjectID

# Combine and save everything: HRV plus higher order spreadsheets

UDF save_results: currently simply called in "Execute" section below

In [ ]:
# Save the combined dataframe as spreadsheet like so

hrv_segment_df.to_excel("/analysis/HRV_metrics.xlsx")

# Save the combined artifact log dataframe as spreadsheet

f_artifacts_log_df.to_excel("/analysis/artifacts_log.xlsx")


# Execute the entire analysis

For each file (fetal and maternal):
- call read_mat_file
- call compute_HRV
- save results to Excel

In [311]:
# Initialize data structures
f_artifacts_log=[]
m_artifacts_log=[]
Uber_fHRV=[]
Uber_mHRV=[]
Uber_higher_fHRV=[]
Uber_higher_mHRV=[]

i=0

# Compute & save into lists

while i<=len(f_peaks_files)-1:   #careful - this assumes equal number of fetal and maternal raw files

    # read the peaks file, trim trailing zeros, artifact correct it, convert to RRIs and return the results 
    f_clean_peaks, m_clean_peaks, f_rri, m_rri, f_clean_peaks_artifacts, m_clean_peaks_artifacts=read_mat_file(f_peaks_files[i],m_peaks_files[i])    
    
    fSubjectID=format(f_peaks_files[i].stem)
    mSubjectID=format(m_peaks_files[i].stem)
    
    f_artifacts_log_i=[fSubjectID,f_clean_peaks_artifacts]
    m_artifacts_log_i=[mSubjectID,m_clean_peaks_artifacts]
        
    #save artifact processing log from each file starting with its real SubjectID
    f_artifacts_log.append(f_artifacts_log_i)
    m_artifacts_log.append(m_artifacts_log_i)
    
    # compute all HRV metrics
    fHRV_final=compute_HRV(f_clean_peaks,f_rri,fSubjectID) 
    mHRV_final=compute_HRV(m_clean_peaks,m_rri,mSubjectID)
    
    # update the UBER df
    Uber_fHRV.append(fHRV_final)  
    Uber_mHRV.append(mHRV_final)
    
    # compute higher_order HRV metrics
    fHRV_higher_final=compute_higher_HRV(fHRV_final,fSubjectID)
    mHRV_higher_final=compute_higher_HRV(mHRV_final,mSubjectID)
    
    # update the UBER_higher_df
    Uber_higher_fHRV.append(fHRV_higher_final)  
    Uber_higher_mHRV.append(mHRV_higher_final)  
    
    i+=1
    
    if i>len(f_peaks_files):  
        break
        
print('Computation completed.')

Computation completed.


In [312]:
# save artifacts logs
df_Uber_f_artifacts = pd.DataFrame.from_records(f_artifacts_log) #edit the name as needed
df_Uber_m_artifacts = pd.DataFrame.from_records(m_artifacts_log) #edit the name as needed

df_Uber_f_artifacts.to_excel('analysis/fHRV_UBER_artifacts_log.xlsx', index=False)
df_Uber_m_artifacts.to_excel('analysis/mHRV_UBER_artifacts_log.xlsx', index=False)

In [313]:
# save HRV results
Uber_fHRV_df=pd.concat(Uber_fHRV)
Uber_fHRV_df.to_excel("analysis/fHRV_metrics.xlsx")

Uber_mHRV_df=pd.concat(Uber_mHRV)
Uber_mHRV_df.to_excel("analysis/mHRV_metrics.xlsx")

Uber_higher_fHRV_df=pd.concat(Uber_higher_fHRV)
Uber_higher_fHRV_df.to_excel("analysis/higher_fHRV_metrics.xlsx")

Uber_higher_mHRV_df=pd.concat(Uber_higher_mHRV)
Uber_higher_mHRV_df.to_excel("analysis/higher_mHRV_metrics.xlsx")